In [1]:
import numpy as np
import matplotlib.pyplot as plt
import Function_SHM_HAILCREEK as FHK
import BSDTA as  BS
from scipy.optimize import minimize
import psycopg2
import pandas as pd

In [2]:
#------------------------------ 1.  Integration Points -----------------------#
N = 6
#------------------------------ 2.  Number of modes --------------------------#
Nm = 2
#------------------------------ 2.  Flag identifier to load cases ------------#
Ld =0
#------------------------------ 3. Semirigid Connection ----------------------#
# Semirigid Nodes
NNI = []
NNJ = []
# Semirigid Material
WW = []
Ei = []
# Flag 0 Run Zero length
flag = 0
#------------------------------ 4. Mass ---------------------------------------#
# Mass in each column simulating the mass of the hopper
Mo = 108333
# Mass in each Bin
MB1 = 0
MB2 = 0
# Mass Distribution
Mass_Case = 1
#------------------------------ 5.  Constants --------------------------------#
gama,g=78.5*10**-6,9800.
#------------------------------ 6. Intial Elastic Model ----------------------#
# Elastic modules 
E,G = 200*10**3,0.25*10**3

In [3]:
#------------------------------ 7. Data base results -------------------------#
def get_data(Q1):
    try:
        con = psycopg2.connect(
        host="db-shm-mincka.ckensqtixcpt.ap-southeast-2.rds.amazonaws.com",
        database='shm_hailcreek',
        user='db_admin',
        password='minckA.2020')

    except psycopg2.DatabaseError as e:

        print(f'Error {e}')
        sys.exit(1)

    cur = con.cursor()

    query = f'''SELECT fre,dampopt,phi FROM "modal_analysis_results" WHERE date1='{Q1}';'''
    Datao = pd.read_sql(query,con)
    Datao.sort_values(by=['fre'], ascending=False,inplace = True)
    data_numpy = Datao.to_numpy()
    fre = data_numpy[:,0]
    dampopt = data_numpy[:,1]
    phi = np.array(Datao['phi'].values.tolist()).transpose()
    return fre,dampopt,phi

In [4]:
#------------------------------ 4. Mass ---------------------------------------#
def Model_updating(N,Nm,E,G,Mbo,Mb1,Mb2,Mass_Case,Ld):
    
    Nodes, conect, idele,Members,Names = FHK.Topology() 
    masa = FHK.Hailcreek(Members,Nodes,conect,idele, E,Mbo,Ld,N,Names,flag,NNI,WW,Ei)
    Ew = FHK.load_cases(Ld,conect,idele,Members,gama)
    dist_b1,dist_b2 = FHK.mass_shapedistribution(Mass_Case)      
    FHK.mass_distirbution(masa,Mbo,Mb1,Mb2,dist_b1,dist_b2)

    freq,U_ops = FHK.Modal_analysis(Nm,0)
    return np.array(freq)

def opti(Freq,phi,N,Nm,E,G,Mbo,Mass_Case,Ld):
    MB1 = 5
    MB2 = 5
    fun = lambda x: Model_updating(N,Nm,E,G,Mbo,10**x[0],10**x[1],Mass_Case,Ld)
    def cost(Freq,x):
        Freq = np.array(Freq)
        cost = np.linalg.norm(Freq-fun(x))
        return cost
    x0 = [MB1,MB2]
    fun2 = lambda x: cost(Freq,x)
    res = minimize(fun2, x0)
    return res.x

In [5]:
from datetime import datetime, timedelta

In [6]:
d1 = datetime(int(2020.), int(12.), int(30.), int(03.), int(35.), int(0.))

In [7]:
fre,dampopt,phi = get_data(d1)

In [8]:
%%time
## Scale factor: change when the real structure is running.
opt = opti(fre[:Nm]/30,phi[:,:Nm],N,Nm,E,G,Mo,Mass_Case,Ld)

CPU times: user 14.9 s, sys: 10.8 ms, total: 14.9 s
Wall time: 15 s


In [9]:
opt

array([5.80941836, 5.80941836])

In [10]:
%%time
FHK.Drawing2database(Nm)

CPU times: user 34.2 ms, sys: 3.98 ms, total: 38.2 ms
Wall time: 44.8 ms
